In [122]:
from Dataset import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import VAE
from Model_Wrapper import Wrapper
import helper
# import helper 

In [123]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [124]:
# !bash download.sh afhq-dataset

In [125]:
# Create a dataset and a DataLoader
transforms = {'val':transforms.Compose([
    transforms.Resize([32, 32]),  # Resize the image first
    transforms.ToTensor(),        # Convert PIL image to tensor
]),
'train':transforms.Compose([
    transforms.Resize([32, 32]),  # Resize the image first
    # transforms.RandomHorizontalFlip(p=0.5),       # Random horizontal flip
    transforms.ToTensor(),        # Convert PIL image to tensor
    # transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize for a single channel
])
}


root_dir = "/home/user/elwakeely1/Cuda_Vision_Lab-/Session5/Solution/data/afhq"

# Initialize dataset
train_dataset = Dataset(root_dir=root_dir, split="train", transform=transforms["train"])
test_dataset = Dataset(root_dir=root_dir, split="val", transform=transforms["val"])


In [126]:
print(f"train size: {train_dataset.__len__()}\ntest size: {test_dataset.__len__()}")

train size: 14630
test size: 1500


In [127]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=8,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=8,pin_memory=True)
print("test_dataset per epoch",next(iter(test_loader))[0].size())

test_dataset per epoch torch.Size([32, 1, 32, 32])


In [128]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [129]:
classes = ["cat","dog","wild"]


In [130]:
def vae_loss_function(recons, target, mu, log_var, lambda_kld=1e-3):
    # """
    #     Combined loss function for joint optimization of 
    #     reconstruction and ELBO
    #     """
        recons_loss = F.mse_loss(recons, target)
        kld = (-0.5 * (1 + log_var - mu**2 - log_var.exp()).sum(dim=1)).mean(dim=0)  # closed-form solution of KLD in Gaussian
        loss = recons_loss + lambda_kld * kld
        return loss, (recons_loss, kld)

In [132]:
vae = VAE.VanillaVAE(in_size=(1,32,32), sizes=[1024, 256, 128, 64, 2], act="ReLU")
vae = vae.to(device)

In [133]:
optimizer = torch.optim.Adam(vae.parameters(), lr=3e-3)
# Decay LR by a factor of 10 after 5 epochs with no improvement
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.1, verbose=True)

In [134]:
writer = helper.new_writer("models","ConvLSTMscratch")
train = Wrapper(model_name="VanillaVAE", model = vae, device = device, criterion = vae_loss_function, optimizer = optimizer,writer=writer,show_progress_bar= True)
train.train(2,train_loader,test_loader)
writer.close()

  0%|          | 0/458 [00:00<?, ?it/s]

Epoch 1 Iter 458: loss 0.04046.: 100%|██████████| 458/458 [00:05<00:00, 79.76it/s] 

Loss/train 0.04011714349065675


Loss/test 0.03626453016508133
Loss/test 0.03252716750857678
Loss/test 3.737362668869343


Epoch 2 Iter 458: loss 0.03252.: 100%|██████████| 458/458 [00:05<00:00, 88.34it/s] 

Loss/train 0.036506052096435336


Loss/test 0.035939591718797986
Loss/test 0.03301672268896661
Loss/test 2.922868804728731
